## JupyterDEVS POC


### Attemping to make an animated matplotlib celldevs plot
See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10,8)
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
%load_ext autoreload
%autoreload 2
from pycdevs.cdppwrapper import CDPPWrapper, Model, DrawlogFailedException
from pycdevs import cdppwrapper as cdppExceptions

In [22]:
cellDevsShape = (40, 40)
modelSourceCodeHeader = """
[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : (${spaceHeight},${spaceWidth})
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : ${neighboursList}
initialvalue : 0
initialCellsValue : sites.val
localtransition : voronoiSite

[voronoiSite]
"""
# TODO: Make some kind of meta-DSL for replacements and formulaes inside CellDEVS model DSL
modelSourceCodeHeader = modelSourceCodeHeader\
    .replace('${spaceHeight}', str(cellDevsShape[0]))\
    .replace('${spaceWidth}', str(cellDevsShape[1]))

Programatically defining **diamond shaped voronoi**

In [23]:
def getNeighbourRule(aNeighbour):
    return "rule : {%s} ${expansionWaveDelay} { (0,0)=0 and %s!=0 and %s!=? }" % (aNeighbour, aNeighbour, aNeighbour)

def getDefaultRule():
    return "rule : {(0,0)} 0 { t }"

# Diamond shaped neighbourhood
neighbours = [
    (0, -2), \
    (-1,-1), (0, -1), (1,-1), \
    (-2, 0), (-1,0), (1,0), (2, 0),\
    (-1,1), (0, 1), (1,1), \
    (0, 2)
]
rules ="""
% Expand center cell value in a 3x3 square neighbourhood
"""
for neighbour in neighbours:
    rules = rules + getNeighbourRule(str(neighbour).replace(' ', '')) + '\n'
rules = rules + getDefaultRule() + '\n'
rules = rules.replace('${expansionWaveDelay}', str(100))

In [24]:
modelSourceCodeHeader = modelSourceCodeHeader.replace('${neighboursList}', \
    " ".join(['voronoiExpansion'+str(neighbour).replace(' ', '') for neighbour in neighbours + [(0,0)]]) \
)

In [25]:
modelSourceCode = modelSourceCodeHeader + rules
simpleVoronoiModel = Model(modelSourceCode, 'voronoiExpansion')

In [26]:
print(modelSourceCodeHeader)


[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : (40,40)
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : voronoiExpansion(0,-2) voronoiExpansion(-1,-1) voronoiExpansion(0,-1) voronoiExpansion(1,-1) voronoiExpansion(-2,0) voronoiExpansion(-1,0) voronoiExpansion(1,0) voronoiExpansion(2,0) voronoiExpansion(-1,1) voronoiExpansion(0,1) voronoiExpansion(1,1) voronoiExpansion(0,2) voronoiExpansion(0,0)
initialvalue : 0
initialCellsValue : sites.val
localtransition : voronoiSite

[voronoiSite]



In [27]:
# TODO: VTime builder wrapper
wrapper = CDPPWrapper(simpleVoronoiModel, '00:01:00:00')
try:
    wrapper.run()
except cdppExceptions.SimulationExecutedButFailedException:
    print(wrapper.getSimulationOutput())

In [28]:
try:
    drawlogNPFilename = wrapper.drawlog('00:00:00:100')
except DrawlogFailedException as e:
    print(e.stderr.decode('ascii'))

In [29]:
loadedMatrix = np.load(drawlogNPFilename)
# loadedMatrix = loadedMatrix.reshape(cellDevsShape)

composedMatrix = np.zeros([cellDevsShape[0], cellDevsShape[1], len(loadedMatrix.files)], dtype=np.double)

for index, npFileName in enumerate(loadedMatrix.files):
    composedMatrix[..., index] = loadedMatrix[ npFileName ].reshape(cellDevsShape)

In [30]:
%%capture
index = 0
fig, ax = plt.subplots()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plottedImage = ax.imshow(composedMatrix[..., index])
stepsCount = composedMatrix.shape[2]

In [31]:
def updateImage(*args):
    global index
    index += 1
    index = index % stepsCount
    plottedImage.set_array(composedMatrix[..., index])
    return plottedImage,
    
simulationAnimation = animation.FuncAnimation(fig, updateImage, interval= 100)

HTML(simulationAnimation.to_html5_video())